## PySide2 OpenGL Texture Tutorial

Welcome to PySide2 OpenGL texture tutorial. 
What is a texture ?

Basically a texture is something you can fill vertex area with.
For the most part this would mean using images to cover up certain areas. 
For example, you have drawn a rectangle, you collate and picture of bricks to it and it becomes a brick wall. You collate a picture of stones to it and it becomes a stone wall. 

If you can use a detailed image then you can create the illusion of such a wall without having to define vertices. You can use 1D, 2D, and 3D textures in opengl. 
We will cover probably the most common case of using 2D textures.

Now let's see the final result before we start looking at the code.

In [16]:
import subprocess

subprocess.run(["python", "app.py"])

CompletedProcess(args=['python', 'app.py'], returncode=0)

As usual the application window is the same as in the previous tutorials.

Let's see the constructor of our gl widget.

In [ ]:
class TextureGL(QOpenGLWidget):
    "Texture loading opengl widget"

    def __init__(self, parent=None):
        "Constructor"
        QOpenGLWidget.__init__(self, parent)
        tutoTutoDir = os.path.dirname(__file__)
        tutoPardir = os.path.join(tutoTutoDir, os.pardir)
        tutoPardir = os.path.realpath(tutoPardir)
        mediaDir = os.path.join(tutoPardir, "media")
        shaderDir = os.path.join(mediaDir, "shaders")
        #
        ############## Diff #####################
        # please do look at the shader code of 
        # the texture. It is a little different
        # than triangle shader.
        availableShaders = ["texture"]
        self.shaders = {
            name: {
                "fragment": os.path.join(shaderDir, name + ".frag"),
                "vertex": os.path.join(shaderDir, name + ".vert")
            } for name in availableShaders
        }
        ############### Diff #####################
        # Notice that we are simply using an image
        # The use of QImage is particularly important
        # since it facilitates a lot of things with 
        # with respect to traditional opengl
        imdir = os.path.join(mediaDir, "images")
        imFName = "im"
        imageFile = os.path.join(imdir, imFName + "0.png")
        print("image file:", imageFile)
        self.imagefile = imageFile
        # Notice that we are using
        self.image = QImage(imageFile).mirrored()
        self.core = "--coreprofile" in QCoreApplication.arguments()

        # opengl data related
        self.context = QOpenGLContext()
        self.program = QOpenGLShaderProgram()
        self.vao = QOpenGLVertexArrayObject()
        self.vbo = QOpenGLBuffer(QOpenGLBuffer.VertexBuffer)
        
        ################ Diff ##############
        # texture is going to be defined
        # afterwards but it is important
        # to define it right now due to its
        # reuse in two different methods
        self.texture = None
        self.indices = np.array([
            0, 1, 3,  # first triangle
            1, 2, 3  # second triangle
        ], dtype=ctypes.c_uint)

        # vertex data of the panel that would hold the image
        self.vertexData = np.array([
            # viewport position || texture coords
            0.5,  0.5,  0.0, 1.0, 1.0,  # top right
            0.5,  -0.5, 0.0, 1.0, 0.0,  # bottom right
            -0.5, -0.5, 0.0, 0.0, 0.0,  # bottom left
            -0.5, 0.5,  0.0, 0.0, 1.0  # top left
        ], dtype=ctypes.c_float)

Mainly there are two differences.
We define a QImage that would be used as the content of the texture.
We also define a texture object to be defined later on during the initialization.

As a side note, see the texture shader in the media `texture.frag` and `texture.vert`.

Let's see the code of `initializeGL`.

In [ ]:
    def initializeGL(self):
        "Initialize opengl "
        print('gl initial')
        print(self.getGlInfo())
        # create context and make it current
        self.context.create()
        self.context.aboutToBeDestroyed.connect(self.cleanUpGl)

        # initialize functions
        funcs = self.context.functions()
        funcs.initializeOpenGLFunctions()
        funcs.glClearColor(1, 0, 1, 1)
        
        # shader
        shaderName = "texture"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        # create shader program
        self.program = QOpenGLShaderProgram(self.context)
        self.program.addShader(vshader)
        self.program.addShader(fshader)

        # bind attribute location
        self.program.bindAttributeLocation("aPos", 0)
        self.program.bindAttributeLocation("aTexCoord", 1)

        # link shader program
        isLinked = self.program.link()
        print("shader program is linked: ", isLinked)

        # activate shader program to set uniform an attribute values
        self.program.bind()
        self.program.setUniformValue('myTexture', 0)

        # vbo
        isVbo = self.vbo.create()
        isVboBound = self.vbo.bind()

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate vbo
        self.vbo.allocate(self.vertexData.tobytes(),
                          floatSize * self.vertexData.size)

        ################## Diff #######################
        # this how pyside handles textures to those
        # who are familiar to opengl it should be self 
        # evident what it does. Notice that this is 
        # how we initialize the texture and set 
        # parameters to it
        self.texture = QOpenGLTexture(QOpenGLTexture.Target2D)
        self.texture.create()
        # new school
        self.texture.bind()
        self.texture.setData(self.image)
        self.texture.setMinMagFilters(QOpenGLTexture.Linear,
                                      QOpenGLTexture.Linear)
        self.texture.setWrapMode(QOpenGLTexture.DirectionS,
                                 QOpenGLTexture.Repeat)
        self.texture.setWrapMode(QOpenGLTexture.DirectionT,
                                 QOpenGLTexture.Repeat)

Now as you can see the main difference is how we set texture.

- We provide the target in opengl. Target2D means GL_TEXTURE_2D
- We set data to it AFTER binding the texture.
- We set other parameters using the related set methods.

Now let's see the drawing loop.

In [ ]:
    def paintGL(self):
        "paint gl"
        funcs = self.context.functions()
        # clean up what was drawn
        funcs.glClear(pygl.GL_COLOR_BUFFER_BIT)

        self.program.bind()
        ############### Diff #################
        self.program.enableAttributeArray(0)
        self.program.enableAttributeArray(1)
        floatSize = ctypes.sizeof(ctypes.c_float)

        # set attribute values
        self.program.setAttributeBuffer(0,  # viewport position
                                        pygl.GL_FLOAT,  # coord type
                                        0,  # offset
                                        3,
                                        5 * floatSize
                                        )
        self.program.setAttributeBuffer(1,  # viewport position
                                        pygl.GL_FLOAT,  # coord type
                                        3 * floatSize,  # offset
                                        2,
                                        5 * floatSize
                                        )
        # bind texture
        self.texture.bind()
        funcs.glDrawElements(pygl.GL_TRIANGLES,
                             self.indices.size, pygl.GL_UNSIGNED_INT,
                             self.indices.tobytes())

By far the most different part about rendering texture is the drawing loop. The main difference is that we set values related to VAO in here rather than in `initializeGL`.

And that's it. Now you know how to load a texture, or how to render an image in opengl. 